In [7]:
library(tidyverse)

In [8]:
data_dir <- file.path("..", "data")

In [9]:
model <- readRDS(file.path(data_dir, "multivariate_model.rds"))

In [10]:
coefs_mtx <- confint(model)

# 这个函数可以统计置信区间。

Waiting for profiling to be done...



In [11]:
coefs <- coefs_mtx %>%
exp %>%
as.data.frame(stringsAsFactors = FALSE) %>%
setNames(c("lo", "hi")) %>%
mutate(cov = rownames(.))

ests <- model %>%
summary %>%
pluck("coefficients") %>%
as.data.frame(stringsAsFactors = FALSE) %>%
setNames(c("est", "se", "zval", "pval")) %>%
mutate(cov = rownames(.)) %>%
mutate(est = exp(est))

df <- coefs %>%
  full_join(ests, by = "cov") %>%
  mutate(ests = sprintf("%s (%s~%s)",
                        round(est, 2), round(lo, 2), round(hi, 2))) %>%
  mutate(cov_ = cov) %>%
  mutate(cov = case_when(
    grepl("po2|pco2", cov) ~ cov,
    TRUE ~ str_replace_all(cov, "[0-9]", "")
  )) %>%
  mutate(cov = str_replace_all(cov, "_", " ")) %>%
  mutate(cov = str_replace_all(cov, "lab|vs|flag|first", "")) %>%
  mutate(cov = str_replace_all(cov, "\\s+$|^\\s+", "")) %>%
  mutate(cov = case_when(
    grepl("lab|day", cov_) ~ toupper(cov),
    TRUE ~ tools::toTitleCase(cov)
  )) %>%
  mutate(cov = case_when(
    cov == "Weight" ~ "Weight (Kg)",
    cov == "Sapsii" ~ "SAPS Score",
    TRUE ~ paste(cov, "(tested)")
  )) %>%
  filter(!grepl("Intercept", cov)) %>%
  mutate(pval = case_when(
    pval < 0.01 ~ "<0.01",
    TRUE ~ as.character(round(pval, 2))
  )) %>%
  dplyr::select(Covariate = cov, `Odds Ratio (2.5%~97.5%)` = ests, `p value` = pval)

print(df)

              Covariate Odds Ratio (2.5%~97.5%) p value
1          Gcs (tested)         0.96 (0.6~1.49)    0.86
2      genderF (tested)        1.48 (1.08~2.05)    0.02
3          Age (tested)        1.02 (1.01~1.03)   <0.01
4            SAPS Score        1.03 (1.02~1.05)   <0.01
5           Weight (Kg)        0.99 (0.98~0.99)   <0.01
6   Sofa Hours (tested)         1.24 (1.18~1.3)   <0.01
7   Heart Rate (tested)           1.01 (1~1.01)    0.05
8          Map (tested)           1 (0.99~1.01)    0.96
9         Temp (tested)         0.9 (0.77~1.06)     0.2
10  HEMOGLOBIN (tested)        0.96 (0.89~1.03)     0.3
11    PLATELET (tested)                 1 (1~1)     0.5
12         WBC (tested)           1 (0.99~1.02)    0.64
13          PH (tested)       1.52 (0.13~19.25)    0.74
14    CHLORIDE (tested)        0.95 (0.91~0.99)   <0.01
15      SODIUM (tested)            1.05 (1~1.1)    0.03
16         BUN (tested)           1.01 (1~1.02)    0.02
17 BICARBONATE (tested)            0.95 (0.9~1) 

In [12]:
data.table::fwrite(df, file.path(data_dir, "coefficients.csv"))